# Preprocesamiento datos Alquiler AMBA

### Importar librerías

#### División del dataset

Agrego datos realcionados a al media del precio del dataset de TRAIN para evitar Data Leakage.
Hago mean encoding para ITE_ADD_NEIGHBORHOOD_NAME

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Cargar el archivo a procesar

In [ ]:
"path_toia = '/Users/victoria/Desktop/alquiler_AMBA_dev.csv'"
path_toia = '/Users/benjavitale/Documents/ML/TP_F/alquiler_AMBA_dev.csv'
df_original = pd.read_csv(path_toia, low_memory=False)

In [ ]:
df = pd.read_csv(path_toia, low_memory=False)
filas, columnas = df.shape
print(f"El dataset tiene {filas} filas y {columnas} columnas.")

### Exploración de los Datos

In [ ]:
df.describe()

In [ ]:
df.info()

#### Datos faltantes por columna

In [ ]:
faltantes = df.isnull().sum()
print(faltantes)

### Limpieza de datos

#### Se eliminan las filas duplicadas

In [ ]:
df = df.drop_duplicates()
print(f"Cantidad de filas después de eliminar duplicadas: {len(df)}")

#### 1. Eliminacion de columnas
Las columnas Longitud, Latitud y id_grid no van a ser necesarias para la implementacion del modelo ya que o contienen informacion redundante o demasiados abstracta para agregar precision de prediccion.

Por otro lado optamos por eliminar la columna STATE_NAME ya que esta es muy parecida a la columna CITY_NAME la cual es mas precisa pero aun asi mostrando valores muy similares. Para no aprender lo mismo sacamos la menos precisa.


In [ ]:
df.drop(columns=['LONGITUDE'], inplace=True)
df.drop(columns=['LATITUDE'], inplace=True)
df.drop(columns=['id_grid'], inplace=True)

#### 2. MesListing
Se cambia a tipo de dato datetime. No hay valores faltantes. Tipo de datos: datetime64

In [ ]:
df['MesListing'] = pd.to_datetime(df['MesListing'])
df['anio'] = df['MesListing'].dt.year
df['mes'] = df['MesListing'].dt.month
df['dia'] = df['MesListing'].dt.day
df = df.drop(columns=['MesListing']) 

#### 3. TIPOPROPIEDAD
Se elimina la columna ya que son todos departamentos y no agrega información.

In [ ]:
df['TIPOPROPIEDAD'].unique()
df.drop(columns=['TIPOPROPIEDAD'], inplace=True)

#### 4. STotalM2
Había 20 valores faltantes. Se eliminaron estas filas ya que no eran representativas. Hay otros 8227 valores que dice 00 que no tiene sentido por lo que se lo reemplaza con la media.

In [ ]:
df = df.dropna(subset=['STotalM2'])
filas, columnas = df.shape
print(f"El dataset tiene {filas} filas y {columnas} columnas.")


In [ ]:
df_ceros = df[df['STotalM2'] == 0]
print(df_ceros)
media_stotalm2 = df.loc[df['STotalM2'] > 0, 'STotalM2'].mean()
df['STotalM2'] = df['STotalM2'].replace(0, media_stotalm2)
print(f"Cantidad de valores igual a cero en STotalM2 después de imputar: {(df['STotalM2'] == 0).sum()}")

#### 5. SConstrM2

In [ ]:
media_stotalm2 = df.loc[df['SConstrM2'] > 0, 'SConstrM2'].mean()
df['SConstrM2'] = df['SConstrM2'].replace(0, media_stotalm2)
print(f"Cantidad de valores igual a cero en STotalM2 después de imputar: {(df['SConstrM2'] == 0).sum()}")


#### 6. Dormitorios
Se borrarlon las filas con dormitorios> 40. Para los que estan entre 40 y 20 se analizó los M2 construídos y si tenía al menos 400 se conservó y sino se eliminó. Se eliminaron las filas con 0 dormitorios y 0 ambientes.

In [ ]:
umbral_superficie = 400
df_filtrar = df[(df['Dormitorios'] >= 20) & (df['Dormitorios'] <= 40)]
a_eliminar = df_filtrar[df_filtrar['STotalM2'] < umbral_superficie]
print(f"Cantidad de filas eliminadas por STotalM2 menor a {umbral_superficie}: {len(a_eliminar)}")
df = df[~((df['Dormitorios'] >= 20) & (df['Dormitorios'] <= 40) & (df['STotalM2'] < umbral_superficie))]
df = df[~((df['Dormitorios'] == 0) & (df['Ambientes'] == 0))]
print(f"Cantidad de filas restantes con Dormitorios == 0 y Ambientes == 0: {len(df[(df['Dormitorios'] == 0) & (df['Ambientes'] == 0)])}")


#### 7. Banos
Se eliminaron las filas que contenian mas de 7 y menos de 0 baños.

In [ ]:
df = df[df['Banos'] <= 7]
df = df[df['Banos'] > 0]

#### 8. Ambientes
Se eliminan las filas en las que ambientes < dormitorios y que ambientes < baños y aquellas con mas de 30 ambientes.

In [ ]:
df = df[~((df['Ambientes'] < df['Dormitorios']) | (df['Ambientes'] < df['Banos']))]
df = df[~((df['Ambientes'] >= 30 )) ]

#### 9. SitioOrigen
Se elimina la columna ya que no aporta información relevamte. 99% de los datos faltantes.

In [ ]:
df.drop(columns=['SitioOrigen'], inplace=True)

#### 10. Antiguedad

Se paso a números. Se eliminaron las filas con valores faltantes. Se eliminaron los valores mayores a 2024 y menores a 0. Para los que parecían ser años se hizo 2024 menos el año para obtener el número de antiguedad. Se paso a columna categorica del 1 al 5 (del más nuevo al más antiguo).

In [ ]:
df = df[df['Antiguedad'].notnull()]
df['Antiguedad'] = df['Antiguedad'].replace(r'[^0-9.]', '', regex=True)
df['Antiguedad'] = pd.to_numeric(df['Antiguedad'], errors='coerce').astype('Int64')
df = df[~((df['Antiguedad'] > 2024) | (df['Antiguedad'] < 0))]

In [ ]:
ano_actual = 2024
es_ano = df['Antiguedad'] >= 1700
df.loc[es_ano, 'Antiguedad'] = ano_actual - df.loc[es_ano, 'Antiguedad']
df['Antiguedad'] = df['Antiguedad'].astype(int)
df = df[df['Antiguedad'] <= 500]

In [ ]:
def categorizar_antiguedad(antiguedad):
    if antiguedad <= 5:
        return 1  
    elif 5 < antiguedad <= 15:
        return 2  
    elif 15 < antiguedad <= 40:
        return 3  
    elif 40 < antiguedad <= 80:
        return 4  
    else:
        return 5  

df['Antiguedad'] = df['Antiguedad'].apply(lambda x: categorizar_antiguedad(x) if pd.notnull(x) else None)

#### 11. Cisterna
Columna eliminada ya que la presencia de una cisterna probablemente tenga poco impacto directo en el precio de la propiedad y hay 58228 datos faltantes.

In [ ]:
df.drop(columns=['Cisterna'], inplace=True)

#### 12. Función que convierte a un valor apropiado las columnas que tienen como valores si, no ,1 ,0. 

In [ ]:
def pasar_binarios(x):
    df[x] = df[x].replace({
        '0.0': 0, 'No': 0, '0': 0, '   0': 0, 
        '1.0': 1, '1': 1, 'Sí': 1
    })
    df[x] = pd.to_numeric(df[x], errors='coerce')
    df[x] = df[x].fillna(0)
    df[x].value_counts()
x = ['AreaJuegosInfantiles','Chimenea','Ascensor','SalonFiestas','Seguridad','Pileta','Cocheras','PistaJogging','EstacionamientoVisitas','Lobby','AreaParrillas','CanchaTennis','AreaCine', 'LocalesComerciales', 'Amoblado','Jacuzzi', 'AccesoInternet','BusinessCenter', 'Gimnasio', 'Laundry', 'Calefaccion', 'SalonDeUsosMul', 'AireAC', 'Recepcion', 'Estacionamiento']
for i in x:
    pasar_binarios(i)

#### 13. SistContraIncendios
Se elimina la columna ya que no tiene mucha info

In [ ]:
df.drop(columns=['SistContraIncendios'], inplace=True)

#### 14. ITE_TIPO_PROD
Se paso a valores 0, 1 y 2.

In [ ]:
df['ITE_TIPO_PROD'] = df['ITE_TIPO_PROD'].map({'N': 1, 'U': 2, 'S': 0})


#### 15. Year
Se elimin la columna porque solo hay años 2022 y 2021


In [ ]:
df.drop(columns=['year'], inplace=True)


#### 16. ITE_ADD_STATE_NAME
One hot para las 4 zonas.
Capital Federal        143041
Bs.As. G.B.A. Norte     43576
Bs.As. G.B.A. Oeste     30029
Bs.As. G.B.A. Sur       23885


In [ ]:
df = pd.get_dummies(df, columns=['ITE_ADD_STATE_NAME'], prefix='state')

#### 17. ITE_ADD_CITY_NAME
Se agruparon las ciudades con menos de 1000 propiedades en la categoria 'otros' y se hizo one-hot.

In [ ]:
threshold = 1000
city_counts = df['ITE_ADD_CITY_NAME'].value_counts()
df['ITE_ADD_CITY_NAME'] = df['ITE_ADD_CITY_NAME'].apply(
    lambda x: x if city_counts[x] >= threshold else 'Otros'
)
df = pd.get_dummies(df, columns=['ITE_ADD_CITY_NAME'], prefix='city')


#### 18. Precios

Se detectó la presencia de outliers y se eliminaron.

In [ ]:
lower_bound = 50000  
upper_bound = 1700000  

df = df[(df['precio_pesos_constantes'] >= lower_bound) & 
        (df['precio_pesos_constantes'] <= upper_bound)]

#### 20. Agrego más features

Más features relacionadas con los M2

In [ ]:
df = df[df['SConstrM2'] <= df['STotalM2']]

df['SNoConstrM2'] = df['STotalM2'] - df['SConstrM2']
df['SConstrRatio'] = df['SConstrM2'] / df['STotalM2']
df['SNoConstrRatio'] = 1 - df['SConstrRatio']

Agrego una columan que contenga todos los servicios juntos.

In [ ]:
x = ['AreaJuegosInfantiles', 'Chimenea', 'Ascensor', 'SalonFiestas', 'Seguridad', 
     'Pileta', 'Cocheras', 'PistaJogging', 'EstacionamientoVisitas', 'Lobby', 
     'AreaParrillas', 'CanchaTennis', 'AreaCine', 'LocalesComerciales', 'Amoblado', 
     'Jacuzzi', 'AccesoInternet', 'BusinessCenter', 'Gimnasio', 'Laundry', 
     'Calefaccion', 'SalonDeUsosMul', 'AireAC', 'Recepcion', 'Estacionamiento']

df['TieneServicios'] = df[x].any(axis=1).astype(int)

In [ ]:
df = df.dropna(subset=['ITE_ADD_NEIGHBORHOOD_NAME'])
y = df['precio_pesos_constantes']

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [ ]:
mean_encoding = X_train.groupby('ITE_ADD_NEIGHBORHOOD_NAME')['precio_pesos_constantes'].mean()
mean_m2 = X_train.groupby('ITE_ADD_NEIGHBORHOOD_NAME')['STotalM2'].mean()

In [ ]:
X_train['mean_precio'] = X_train['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_encoding)
X_train['mean_m2'] = X_train['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_m2)
X_train['mean_p/m2'] = X_train['mean_precio'] / X_train['mean_m2']
X_test['mean_precio'] = X_test['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_encoding)
X_test['mean_m2'] = X_test['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_m2)
X_test['mean_p/m2'] = X_test['mean_precio'] / X_test['mean_m2']

In [ ]:
X_train.drop(columns=['mean_precio'], inplace=True)
X_train.drop(columns=['mean_m2'], inplace=True)
X_test.drop(columns=['mean_precio'], inplace=True)
X_test.drop(columns=['mean_m2'], inplace=True)
X_train.drop(columns=['ITE_ADD_NEIGHBORHOOD_NAME'], inplace=True)
X_test.drop(columns=['ITE_ADD_NEIGHBORHOOD_NAME'], inplace=True)
X_train.drop(columns = ['precio_pesos_constantes'], inplace=True)
X_test.drop(columns = ['precio_pesos_constantes'], inplace=True)

Elimino cualquier fila que contenga un valor faltante. Unicamente de chequeo

In [ ]:
def eliminar_filas_con_nan(X_train, X_test, y_train, y_test):
    """
    Elimina filas con valores NaN de los conjuntos X_train, X_test, y_train, y_test.
    Asegura que las eliminaciones sean consistentes entre X e y.

    Parámetros:
        X_train (pd.DataFrame): Conjunto de entrenamiento de features.
        X_test (pd.DataFrame): Conjunto de prueba de features.
        y_train (pd.Series): Conjunto de entrenamiento de labels.
        y_test (pd.Series): Conjunto de prueba de labels.

    Retorna:
        tuple: X_train, X_test, y_train, y_test limpios.
    """
    train = pd.concat([X_train, y_train], axis=1)
    train = train.dropna()
    X_train = train.iloc[:, :-1]  
    y_train = train.iloc[:, -1]   

    test = pd.concat([X_test, y_test], axis=1)
    test = test.dropna()
    X_test = test.iloc[:, :-1]  
    y_test = test.iloc[:, -1]   

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = eliminar_filas_con_nan(X_train, X_test, y_train, y_test)

## Una vez finalizado el Preprocesamiento lo paso a CSV

In [543]:
#df.to_csv('/Users/victoria/Desktop/alquiler_procesado.csv', index=False)

In [544]:
X_train.to_csv('/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_Xtrain.csv', index=False)
X_test.to_csv('/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_Xtest.csv', index=False)

y_train.to_csv('/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_ytrain.csv', index=False)
y_test.to_csv('/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_ytest.csv', index=False)